In [3]:
import sys
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from datasets import load_dataset
from typing import Dict, List, Tuple
from joblib import Parallel, delayed
from tqdm import tqdm
import os

project_root = os.path.abspath(os.path.join(os.path.dirname("__file__"), '..', '..'))
sys.path.append(project_root)
from baseline.utils.setup import setup_experiment_dir
exp_dir = setup_experiment_dir('local_rf')

In [4]:
def prepare_data(dataset_name):
    dataset = load_dataset(dataset_name)
    data = dataset['data']

    METADATA_COLUMNS = {
        'problem', 'samples', 'solution', 'instruction', 'type', 'level', 'answer_correct', 'extracted_answers',
        'problem_idx', 'solution_idx'
    }
    
    # get first row (problem) to identify columns
    first_item = data[0]
    all_columns = set(first_item.keys())
    
    # First identify judge columns (they're special case with T/F values)
    judge_columns = [col for col in all_columns 
                    if col.startswith('judge_') and col not in METADATA_COLUMNS]
    
    # Then identify numeric feature columns (excluding judge columns)
    numeric_columns = [col for col in all_columns 
                      if col not in METADATA_COLUMNS and 
                      col not in judge_columns and
                      isinstance(first_item[col][0], (int, float, np.number))]
    
    # 1st pass: collect all values for each verdict column
    verdict_values = {col: [] for col in judge_columns}
    for problem in data:
        num_samples = len(problem['samples'])
        for col in judge_columns:
            # Only collect values if they match the number of samples
            if len(problem[col]) == num_samples:
                verdict_values[col].extend([v for v in problem[col] if v is not None])
    
    # calc mode for each verdict column with safety check
    modes = {}
    for col, values in verdict_values.items():
        if not values:  # If all values were None
            modes[col] = False  # default to False for empty columns
        else:
            modes[col] = max(set(values), key=values.count)
    
    # score cols are numeric columns that end with "_score(s)"
    score_columns = [col for col in numeric_columns
                    if col.endswith('_score') or col.endswith('_scores')]
    
    # other numerical columns (e.g. rewards, steps)
    other_num_columns = [col for col in numeric_columns
                        if col not in score_columns]
    
    data_rows = []
    for idx in range(len(data)):
        problem = data[idx]
        num_samples = len(problem['samples'])
        
        # filter columns to only those that match the number of samples/generations
        valid_judge_columns = [col for col in judge_columns if len(problem[col]) == num_samples]
        valid_score_columns = [col for col in score_columns if len(problem[col]) == num_samples]
        valid_other_columns = [col for col in other_num_columns if len(problem[col]) == num_samples]
        
        normalized_scores = normalize_scores(problem, valid_score_columns)
        normalized_nums = normalize_scores(problem, valid_other_columns)
        
        for i in range(num_samples):
            if i < len(problem['answer_correct']):  # check if we have a corresponding answer
                row = {
                    'problem_idx': idx,
                    'solution_idx': i,
                    'is_correct': problem['answer_correct'][i],
                    # Handle judge columns separately (no normalization)
                    **{column: modes[column] if problem[column][i] is None
                       else problem[column][i] for column in valid_judge_columns},
                    # Normalize numeric columns
                    **{column: normalized_scores[column][i] for column in valid_score_columns},
                    **{column: normalized_nums[column][i] for column in valid_other_columns}
                }
                data_rows.append(row)
            
    df = pd.DataFrame(data_rows)
    final_feature_columns = [col for col in (judge_columns + numeric_columns) 
                            if col in df.columns]
    
    return df, final_feature_columns

def normalize_scores(problem, columns):
    """Normalize scores within each problem"""
    normalized = {}
    for col in columns:
        values = np.array(problem[col])
        min_val = np.min(values)
        max_val = np.max(values)
        score_range = max_val - min_val
        if score_range == 0:
            normalized[col] = [0.5] * len(values)
        else:
            normalized[col] = (values - min_val) / score_range
    return normalized


def get_feature_thresholds(rf_model, feature_columns) -> Dict:
    """Get median thresholds for each feature from the RF model"""
    if rf_model is None:
        return {}

    feature_thresholds = {}
    for tree in rf_model.estimators_:
        for feature_idx, threshold in zip(tree.tree_.feature, tree.tree_.threshold):
            if feature_idx >= 0:
                feature_name = feature_columns[feature_idx]
                if feature_name not in feature_thresholds:
                    feature_thresholds[feature_name] = []
                feature_thresholds[feature_name].append(threshold)

    median_thresholds = {
        feature: np.median(thresholds)
        for feature, thresholds in feature_thresholds.items()
    }

    return median_thresholds

In [5]:
def calculate_metrics(df_test, y_pred, y_pred_proba, rf_model, feature_columns):
    """Calculate metrics for a single problem's local RF model"""
    # Generation accuracy
    generation_accuracy = accuracy_score(df_test['is_correct'], y_pred)
    
    # Selection metrics
    prob_proba = y_pred_proba
    prob_labels = df_test['is_correct'].values
    
    # Select solution with highest probability
    selected_idx = np.argmax(prob_proba)
    
    # Calculate base metrics
    tp = 1 if prob_labels[selected_idx] else 0
    fp = 1 if not prob_labels[selected_idx] else 0
    fn = 1 if sum(prob_labels) > 0 and not prob_labels[selected_idx] else 0
    tn = 1 if sum(prob_labels) == 0 and not prob_labels[selected_idx] else 0
    
    # Calculate derived metrics
    selection_accuracy = tp  # Will be 1 or 0 for a single problem
    selection_precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    selection_recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    selection_f1 = 2 * (selection_precision * selection_recall) / (
        selection_precision + selection_recall) if (selection_precision + selection_recall) > 0 else 0
    
    return {
        'problem_idx': df_test['problem_idx'].iloc[0],
        'generation_accuracy': generation_accuracy,
        'selection_accuracy': selection_accuracy,
        'selection_precision': selection_precision,
        'selection_recall': selection_recall,
        'selection_f1': selection_f1,
        'selection_tp': tp,
        'selection_tn': tn,
        'selection_fp': fp,
        'selection_fn': fn,
        'feature_thresholds': get_feature_thresholds(rf_model, feature_columns)
    }

In [6]:
def train_problem_rf(problem_data: Tuple[pd.DataFrame, float, List[str]]) -> Dict:
    """Train and evaluate RF model for a single problem"""
    df_prob, train_percentage, feature_columns = problem_data
    
    X = df_prob[feature_columns].copy()
    y = df_prob['is_correct']
    
    n_positive = y.sum()
    
    # Case 1: insufficient positive examples - predict majority class
    if n_positive <= 3:
        majority_class = int(y.mean() >= 0.5)
        y_pred = np.full(len(y), majority_class)
        y_pred_proba = np.full(len(y), float(majority_class))
        metrics = calculate_metrics(df_prob, y_pred, y_pred_proba, None, feature_columns)
        metrics['insufficient_data'] = True
        return metrics
    
    # Case 2: sufficient data for training
    skf = StratifiedKFold(n_splits=max(2, int(np.ceil(1/train_percentage))),
                         shuffle=True, random_state=42)
    train_idx, test_idx = next(skf.split(X, y))
    
    X_train = X.iloc[train_idx]
    y_train = y.iloc[train_idx]
    X_test = X.iloc[test_idx]
    y_test = y.iloc[test_idx]
    df_test = df_prob.iloc[test_idx]
    
    # Fixed RF parameters
    rf = RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        min_samples_split=10,
        min_samples_leaf=5,
        max_features='sqrt',
        class_weight='balanced',
        random_state=42,
        n_jobs=-1
    )
    
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    y_pred_proba = rf.predict_proba(X_test)[:, 1]
    
    metrics = calculate_metrics(df_test, y_pred, y_pred_proba, rf, feature_columns)
    metrics['insufficient_data'] = False
    return metrics

In [7]:
def aggregate_metrics(problem_metrics: List[Dict]) -> Dict:
    """Aggregate metrics across all problems"""
    return {
        'generation_accuracy': np.mean([m['generation_accuracy'] for m in problem_metrics]),
        'selection_accuracy': np.mean([m['selection_accuracy'] for m in problem_metrics]),
        'selection_precision': np.mean([m['selection_precision'] for m in problem_metrics]),
        'selection_recall': np.mean([m['selection_recall'] for m in problem_metrics]),
        'selection_f1': np.mean([m['selection_f1'] for m in problem_metrics]),
        'selection_tp': sum(m['selection_tp'] for m in problem_metrics),
        'selection_tn': sum(m['selection_tn'] for m in problem_metrics),
        'selection_fp': sum(m['selection_fp'] for m in problem_metrics),
        'selection_fn': sum(m['selection_fn'] for m in problem_metrics),
        'total_problems': len(problem_metrics),
        'insufficient_data_problems': sum(1 for m in problem_metrics if m.get('insufficient_data', False))
    }

In [8]:
def train_local_rfs(train_percentage: float):
    """Train and evaluate local RF models in parallel"""
    print(f"\nTraining with {train_percentage*100}% of problems")
    
    df, feature_columns = prepare_data("hazyresearch/CodeContests_Llama_70B_with_LM_Judges_and_RMs_v1")
    
    problem_groups = [
        (group, train_percentage, feature_columns)
        for _, group in df.groupby('problem_idx')
    ]
    
    print("Training models for each problem...")
    problem_metrics = Parallel(n_jobs=-1, verbose=1)(
        delayed(train_problem_rf)(problem_data)
        for problem_data in problem_groups
    )
    
    aggregated_metrics = aggregate_metrics(problem_metrics)
    
    problem_metrics_df = pd.DataFrame(problem_metrics)
    problem_metrics_df.to_csv(
        os.path.join(exp_dir, 'per_problem', f'metrics_{train_percentage}.csv'),
        index=False
    )
    
    return aggregated_metrics

In [9]:
def plot_results(results_df):
    """Plot accuracy metrics"""
    plt.figure(figsize=(10, 5))
    
    # Selection@1 plot
    plt.subplot(1, 2, 1)
    plt.plot(results_df['train_percentage'] * 100,
             results_df['selection_accuracy'],
             '-o', label='Local RF')
    plt.xlabel('Percentage of Training Data')
    plt.ylabel('Selection@1')
    plt.title('Selection@1 vs Training Data Size')
    plt.grid(True)
    plt.ylim(0, 1)
    plt.legend()

    # Generation Accuracy plot
    plt.subplot(1, 2, 2)
    plt.plot(results_df['train_percentage'] * 100,
             results_df['generation_accuracy'],
             '-o', label='Local RF')
    plt.xlabel('Percentage of Training Data')
    plt.ylabel('Generation Accuracy')
    plt.title('Generation Accuracy vs Training Data Size')
    plt.grid(True)
    plt.ylim(0, 1)
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(os.path.join(exp_dir, 'plots.png'))
    plt.close()

print("Starting local RF training pipeline...")
train_percentages = [0.1, 0.3, 0.5, 0.7]
results = []

for train_percentage in train_percentages:
    metrics = train_local_rfs(train_percentage)
    metrics['train_percentage'] = train_percentage
    results.append(metrics)
    
    print(f"\nResults for {train_percentage*100}% training data:")
    print(f"Selection accuracy: {metrics['selection_accuracy']:.3f}")
    print(f"Generation accuracy: {metrics['generation_accuracy']:.3f}")
    print(f"Selection F1: {metrics['selection_f1']:.3f}")
    print(f"Insufficient data problems: {metrics['insufficient_data_problems']}")

results_df = pd.DataFrame(results)
results_df.to_csv(os.path.join(exp_dir, 'metrics.csv'), index=False)

plot_results(results_df)

Starting local RF training pipeline...

Training with 10.0% of problems
Training models for each problem...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
/Users/brendanmclaughlin/venvs/agents/lib/python3.12/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 8 members, which is less than n_splits=10.
  warnings.warn(
/Users/brendanmclaughlin/venvs/agents/lib/python3.12/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:    2.0s
/Users/brendanmclaughlin/venvs/agents/lib/python3.12/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=10.
  warnings.warn(
/Users/brendanmclaughlin/venvs/agents/lib/python3.12/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 7 members, which is less than n_splits=10.
  

IndexError: index 1 is out of bounds for axis 1 with size 1